In [9]:
import tensorflow as tf
from tensorflow.keras import backend as K
import numpy as np
import h5py
from random import shuffle
from functools import reduce
import matplotlib.pyplot as plt
# !pip install tensorflow==2.0.0-beta1

In [10]:
max_seg = 12
max_word = 34
level_class_cnt = 3
test_percentage = 0.2

dropout_rate = 0.5
hidden_feature_dim = 70
gru_feature_dim = 150
kernel_heights = [3, 4, 5]

batch_size = 256
epochs = 25

w2v_weights_path = '/Volumes/CCsChunk2/datasets/nlp/milnet/nn_input/1/weights.npy'
tensorboard_log_dir = '/Users/Frost/Desktop/log/'
input_path = '/Volumes/CCsChunk2/datasets/nlp/milnet/nn_input/1/electronics.hdf5'

sample_amount = 0
mini_batch_cnt = 0
with h5py.File(input_path) as in_file:
    for index in range(len(in_file['label/'].keys())):
        mini_batch_cnt += 1
        sample_amount += len(in_file['label/' + str(index)])
batch_indices = [*range(mini_batch_cnt)]
shuffle(batch_indices)
train_batches = batch_indices[0:int(mini_batch_cnt * (1 - test_percentage))]
test_batches = batch_indices[int(mini_batch_cnt * (1 - test_percentage)):]

In [14]:
w2v = np.load(w2v_weights_path, allow_pickle=True)
w2v_len = w2v.shape[1]

def label_map(raw_label):
    if raw_label < 2:
        return 0
    elif raw_label < 3:
        return 1
    else:
        return 2

def data_generator(batch_indices, epochs=epochs):
    global batch_size, input_path
    with h5py.File(input_path) as in_file:
        feature_array, label_array = np.zeros((batch_size, max_seg, max_word)), np.zeros((batch_size, 1))
        batch_index = 0
        for _ in range(epochs):
            shuffle(batch_indices)
            for index in batch_indices:
                doc, label = in_file['document/' + str(index)], in_file['label/' + str(index)]
                random_doc_order = [*range(len(doc))]
                shuffle(random_doc_order)
                for i in random_doc_order:
                    feature_array[batch_index] = np.array([doc[i].astype('float64')])
                    label_array[batch_index] = label_map(label[i])
                    batch_index += 1
                    if batch_index == batch_size:
                        yield feature_array, label_array
                        batch_index = 0
                        feature_array, label_array = np.zeros((batch_size, max_seg, max_word)), np.zeros((batch_size, 1))

In [12]:
''' Slice a piece from one dimension.

The layer would slice the `index`th dimension from `target_dim` dimension of
the input tensor, which have `total_dim` dimensions, then squeeze the tensor
over the sliced dimension.

Args:
    total_dim (int): The total number of dimensions of the input tensor.
    target_dim (int): The index of the dimension that need to slice.
    index (int): The index of the dimension to keep in the slicing operation.

Returns:
    (Model): A keras model that implement the operation.
'''
def __get_filter_layer(total_dim, target_dim, index):
    def tensor_filter(tensor_in):
        nonlocal index
        begin = [0 if i != target_dim else index for i in range(total_dim)]
        size = [-1 if i != target_dim else 1 for i in range(total_dim)]
        return tf.squeeze(tf.slice(tensor_in, begin, size), axis=target_dim)
    return tf.keras.models.Sequential([
        tf.keras.layers.Lambda(tensor_filter)
    ])


''' Implement `submodel` for each slice of tensor.

The model would slice its input tensor into pieces using `__get_filter_layer` 
along `branch_index`th dimension, then for each slice, implement submodel, 
finally the outputs of different submodels would be concated and reshaped to 
meet the demand of output.

Args:
    input_shape tuple(int): The shape of the input tensor.
    branch_index (int): The index of the dimension to slice, start from 0 as 
        sample amount dimension.
    output_shape tuple(int): The shape of the output tensor.
    submodel (Model): The model to apply to different slices.
    args (dict): The argument dictionary for `submodel`.
'''
def __get_branch_model(input_shape, branch_index, output_shape, submodel, args={}):
    model_input = tf.keras.Input(input_shape)
    sliced_inputs = [__get_filter_layer(len(input_shape) + 1, branch_index, i)(model_input) 
                     for i in range(input_shape[branch_index - 1])]
    sub_instance = submodel(**args)
    branch_models = [sub_instance(sliced_inputs[i]) 
                     for i in range(input_shape[branch_index - 1])]
    concated_layers = tf.keras.layers.Concatenate()(branch_models)
    model_output = tf.keras.layers.Reshape(output_shape)(concated_layers)
    return tf.keras.Model(model_input, model_output)


''' A CNN unit to encode segment with single kernel height.

The unit would apply a convolution to its input to get a 2-dimensional 
tensor, then apply max overtime pooling to get a single dimensional tensor.

Args:
    input_shape ((int, int)): The shape of segment matrix. (word_max, w2v_len)
    kernel_height (int): The height of the convolution kernel.
    index (int): The index of the segment in its belonging document.

Returns:
    (Model): The CNN model to encode the segment matrix.
'''
def __get_sentence_encode_unit(input_shape, kernel_height):
    global w2v_len
    cnned_height = input_shape[0] - kernel_height + 1
    return tf.keras.models.Sequential([
        tf.keras.layers.Reshape((*input_shape, 1)),
        tf.keras.layers.Conv2D(hidden_feature_dim, (kernel_height, w2v_len)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU(),
        tf.keras.layers.Reshape((cnned_height, hidden_feature_dim, 1)),
        tf.keras.layers.MaxPool2D((cnned_height, 1))
    ])


''' A CNN unit to encode segment with multiple kernel heights

The unit would apply operation defined in `__get_sentence_encode_unit` for 
different kernel heights, then concat the result as a 1-dimensional tensor.

Args:
    input_shape ((int, int)): The shape of the document. (word_max, w2v_len)
    kernel_heights ([int]): The list of the kernel heights.
    index: The index of the segment in its belonging document.

Returns:
    (Model): The CNN model to encode the segment matrix.
'''
def __get_multi_kernel_encode_unit(input_shape, kernel_heights):
    global w2v_len
    model_input = tf.keras.Input(input_shape)
    cnn_layers = [__get_sentence_encode_unit((input_shape), h)
                     (model_input) for h in kernel_heights]
    concated_layers = tf.keras.layers.concatenate(cnn_layers)
    model_output = tf.keras.layers.Flatten()(concated_layers)
    return tf.keras.Model(model_input, model_output)


''' The softmax linear classifier for predicting segment sentiment.

Returns:
    (Model): The softmax linear classifier to predict segment sentiment.
'''
def __get_seg_classifier_unit():
    return tf.keras.models.Sequential([
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(level_class_cnt, activation='softmax')
    ])


''' The unit to get the attention weight for a segment from hidden feature.

Returns:
    (Model): The model for predicting attention weight for a segment.

'''
def __get_attention_unit():
    return tf.keras.models.Sequential([
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(2 * gru_feature_dim, activation='tanh'),
        tf.keras.layers.Dense(1, use_bias=False, activation='softmax')
    ])


''' A bidirectional-GRU unit to extract the hidden vectors.

The hidden vectors are used to predict the attention weights of the model.

Returns:
    (Model): The bidirectional-GRU unit to predict the hidden vectors.
'''
def __get_bidirectional_gru_unit():
    return tf.keras.models.Sequential([
        tf.keras.layers.Bidirectional(
            tf.keras.layers.GRU(gru_feature_dim, return_sequences=True)
        )
    ])


def performance_judge(model, generator, class_cnt=level_class_cnt):
    eps = np.finfo(float).eps
    accuracy, precisions, recalls, f1s = [], [], [], []
    for i, (features, labels) in enumerate(generator):
        predicted = model.predict(features)
        precisions.append([])
        recalls.append([])
        f1s.append([])
        contingency_table = np.zeros((class_cnt, class_cnt))
        for index in range(features.shape[0]):
            contingency_table[int(labels[index][0])][np.argmax(predicted[index])] += 1
        accuracy.append(np.trace(contingency_table) / features.shape[0])
        for index in range(class_cnt):
            pass
            precisions[i].append(contingency_table[index][index] / (np.sum(contingency_table[:, index]) + eps))
            recalls[i].append(contingency_table[index][index] / (np.sum(contingency_table[index, :]) + eps))
            f1s[i].append(2 * precisions[i][-1] * recalls[i][-1] / ((precisions[i][-1] + recalls[i][-1]) + eps))
    precisions = [float(sum(l))/len(l) for l in zip(*precisions)]
    recalls = [float(sum(l))/len(l) for l in zip(*recalls)]
    f1s = [float(sum(l))/len(l) for l in zip(*f1s)]
    print('Accuracy:', round(reduce(lambda x, y: x + y, accuracy) / len(accuracy), 3))
    for index in range(class_cnt):
        print('_____ Class', index, '_____')
        print('Precision\t', round(precisions[index], 3))
        print('Recall\t\t', round(recalls[index], 3))
        print('F1 Score\t', round(f1s[index], 3))

In [13]:
print('Constructing Model ...')

model_input = tf.keras.Input((max_seg, max_word))

embedding_layer = tf.keras.layers.Embedding(
    input_dim=w2v.shape[0], 
    output_dim=w2v_len, 
    weights=[w2v], 
    input_length=max_word, 
    trainable=False
)(model_input)

encoding_model = __get_branch_model(
    input_shape=(max_seg, max_word, w2v_len), 
    branch_index=1, 
    output_shape=(max_seg, len(kernel_heights) * hidden_feature_dim), 
    submodel=__get_multi_kernel_encode_unit, 
    args={'kernel_heights': kernel_heights, 'input_shape': (max_word, w2v_len)}
)(embedding_layer)

biglu_model = __get_bidirectional_gru_unit()(encoding_model)

attention_model = __get_branch_model(
    input_shape=(max_seg, 2 * gru_feature_dim), 
    branch_index=1, 
    output_shape=(max_seg, 1), 
    submodel=__get_attention_unit
)(biglu_model)

classification_model = __get_branch_model(
    input_shape=(max_seg, len(kernel_heights) * hidden_feature_dim), 
    branch_index=1, 
    output_shape=(max_seg, level_class_cnt), 
    submodel=__get_seg_classifier_unit
)(encoding_model)

weighted_layer = tf.keras.layers.Multiply()([attention_model, classification_model])

reduce_layer = tf.keras.layers.Lambda(tf.reduce_mean, arguments={'axis': 1})(weighted_layer)

model = tf.keras.Model(model_input, reduce_layer)

print('Model Constructed. Compiling ...')

model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

print('Model Compiled.')

model.summary()

Constructing Model ...
Model Constructed. Compiling ...
Model Compiled.
Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 20, 30)]     0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 20, 30, 300)  58749600    input_1[0][0]                    
__________________________________________________________________________________________________
model_1 (Model)                 (None, 20, 210)      253050      embedding[0][0]                  
__________________________________________________________________________________________________
sequential_23 (Sequential)      (None, 20, 300)      325800      model_1[1][0]                    
____________________

In [ ]:
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=tensorboard_log_dir + "{}".format(time()))

model.fit_generator(
    data_generator(train_batches), 
    steps_per_epoch=(sample_amount * (1 - test_percentage) // batch_size) - 1,
    epochs=epochs
)

# model.save(model_out_path)

In [ ]:
print('############ Test Error ############')
performance_judge(model, data_generator(test_batches, epochs=1))

print('########## Training Error ##########')
performance_judge(model, data_generator(train_batches, epochs=1))